In [ ]:
from IPython.utils import io
import torch
import cv2
import PIL
import imageio
import pickle
import os
import numpy as np
import random
import ipywidgets as widgets
import matplotlib.pyplot as plt
from PIL import Image
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte
from ipywidgets import fixed

In [ ]:
%cd /home/taki/ganspace

### Load model

In [ ]:
# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  layer='style',
  output_class='egg',
  components=80,
  use_w=True,
  n=1_000_000,
  batch_size=10_000, # style layer quite small
)

# config = Config(
#   model='StyleGAN2',
#   layer='style',
#   output_class='ffhq',
#   components=80,
#   use_w=True,}
#   n=1_000_000,
#   batch_size=10_000, # style layer quite small
# )


inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model
with open(str(os.path.split(path_to_components)[0]) + '/pca_model.pkl', mode='rb') as f:
    transformer = pickle.load(f)

named_directions = {} #init named_directions dict to save directions

### Load components

In [ ]:
comps = np.load(path_to_components)
lst = comps.files
latent_dirs = [] # shape:(80, 512)
latent_stdevs = []
lat_stdevs = []

load_activations = True

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
      if item == 'act_mean':
        comp_mean = comps[item]
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          lat_stdevs.append(comps[item][i])

    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])

num = 5
comp_dir = [] # shape:(num, 512)
comp_dir_stdev = []


for i in range(num):
    comp_dir.append(latent_dirs[i])
    comp_dir_stdev.append(latent_stdevs[i])
    
print(f'Loaded Component No. 1~{num}')

In [ ]:
latent_stdevs[0:5]

In [ ]:
lat_stdevs[0:5]

In [ ]:
M = 10
seed = 0
latents = model.sample_latent(M, seed=seed).cpu().numpy()

In [ ]:
comp_mean[0]

### Generate parameter and images

In [ ]:
M = 10
seed = 0

# latents = model.sample_latent(M, seed=seed).cpu().numpy() # 元の点をランダム生成
w = np.array([comp_mean[0].copy() for _ in range(M)]) # 潜在空間の平均座標をM個用意
#w = np.empty((latents.shape[0], latents.shape[1]), dtype=np.float32) # これに書き込んでいく
#w_dir = np.array(latent_dirs) # 
w_dir = np.array(comp_dir) # 

for i in range(M):
    for j in range(num):
        #w[i:i+1] = w[i:i+1]+random.uniform(-10, 10)*w_dir[j] # latentの元の点に、ランダムに選んだベクトル方向*ランダムスカラー値を足す
        w[i:i+1] = w[i:i+1]+random.uniform(-3*comp_dir_stdev[j], 3*comp_dir_stdev[j])*w_dir[j] # latentの元の点に、ランダムに選んだベクトル方向*ランダムスカラー値を足す
    out = model.sample_np(w[i:i+1])
    img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
    #img.save(f'/home/taki/ganspace/gen_img_random-to-comp_-3to3sigma_batch64x4_310k-iter_fid=38-30437/{i:05}.png')

param = transformer.transform(w)

print(w.shape)
print(param.shape)

# np.save('/home/taki/ganspace/gen_img_random-to-comp_-3to3sigma_batch64x4_310k-iter_fid=38-30437/w', w)
# np.save('/home/taki/ganspace/gen_img_random-to-comp_-3to3sigma_batch64x4_310k-iter_fid=38-30437/param', param)

In [ ]:
w_dir

In [ ]:
transformer

In [ ]:
param_read.shape

In [ ]:
w_read     = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/w.npy')
param_read = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/param.npy')
print('w')
print(w_read)
print('param')
print(param_read)

In [ ]:
w_read[0]

In [ ]:
param_read[1]

In [ ]:
param_read[2]

In [ ]:
param_read[:,:5][0]

In [ ]:
transformer.inverse_transform(param_read[0])

In [ ]:
np.save('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/pca_score', param_read[:,:5])

In [ ]:
pca_score = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/pca_score.npy')
pca_score

#### 生成画像のファイル名に各次元のスコアを命名

In [ ]:
pca_score = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/pca_score.npy')
pca_score

In [ ]:
source_folder = "/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5"
image_files = [f for f in os.listdir(source_folder) if f.endswith((".jpg", ".jpeg", ".png", ".gif"))]
image_files.sort()

for i, img_file in enumerate(image_files):
    source_path = os.path.join(source_folder, img_file)
    pca_txt = ""
    for score in pca_score[i]:
        if score == pca_score[i][-1]:
            pca_txt += f"{score}"
        else:
            pca_txt += f"{score},"
    new_path = f"{img_file.split('.')[0]}_[{(pca_txt)}].png"
    target_path = os.path.join(source_folder, new_path)
    os.rename(source_path, target_path)
    # print(source_path)
    # print(target_path)

In [ ]:
# image_files[0]をピリオドで分割する

image_files[0].split('.')

### Load images

In [ ]:
images = []
for i in range(0, 1000):
    image = cv2.imread('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/{0:05d}.png'.format(i))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    images.append(image)

images  = np.array(images)
print(images.shape)

In [ ]:
images[0].shape

In [ ]:
w_read[0:1].shape

In [ ]:
w_read[0:1]

In [ ]:
w_read[12:13]

In [ ]:
out = model.sample_np(w_read[0:1])
img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
axs.imshow(img)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
axs.imshow(images[13])

## グリッド表示のための画像を保存

In [ ]:
w_read     = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/w.npy')
param_read = np.load('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/param.npy')
print('w')
print(w_read.shape)
print(w_read)
print('param')
print(param_read.shape)
print(param_read)

In [ ]:
param_read

In [ ]:
param_read[0].shape

In [ ]:
pca_x = param_read
pca_x[0] = 0

pca_x_0 = transformer.inverse_transform(pca_x[0]).astype(np.float32)

out = model.sample_np(pca_x_0.reshape(1,512))
img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
axs.imshow(img)

In [ ]:
pca_x[0]

In [ ]:
pca_x = param_read
pca_x[0] = 0

In [ ]:
param_read

In [ ]:
pca_x[0]

In [ ]:
conb_list = [[0,1], [0,2], [0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
for conb in conb_list:
    comp_num1 = conb[0]
    comp_num2 = conb[1]

    if comp_num1 == 0:
        a=10
    elif comp_num1 == 1 or comp_num1 == 2:
        a=8
    elif comp_num1 == 3:
        a=6
    elif comp_num1 == 4:
        a=5

    
    if comp_num2 == 0:
        b=10
    elif comp_num2 == 1 or comp_num2 == 2:
        b=8
    elif comp_num2 == 3:
        b=6
    elif comp_num2 == 4:
        b=5
    print(conb)
    print(a, b)
    print("")

In [ ]:
pca_x = param_read
pca_x[0] = 0

conb_list = [[0,1], [0,2], [0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
for conb in conb_list:
    comp_num1 = conb[0]
    comp_num2 = conb[1]
    print(comp_num1, comp_num2)

    # Create an array with values ranging from -10 to 10, divided equally into 8 segments
    if comp_num1 == 0:
        first_component = np.linspace(-10, 10, 9)
    elif comp_num1 == 1 or comp_num1 == 2:
        first_component = np.linspace(-8, 8, 9)
    elif comp_num1 == 3:
        first_component = np.linspace(-6, 6, 9)
    elif comp_num1 == 4:
        first_component = np.linspace(-5, 5, 9)

    # Create an array with values ranging from -8 to 8, divided equally into 8 segments
    if comp_num2 == 0:
        second_component = np.linspace(-10, 10, 9)
    elif comp_num2 == 1 or comp_num2 == 2:
        second_component = np.linspace(-8, 8, 9)
    elif comp_num2 == 3:
        second_component = np.linspace(-6, 6, 9)
    elif comp_num2 == 4:
        second_component = np.linspace(-5, 5, 9)


    images = []
    x_list = []


    # ディレクトリを作成
    save_dir = f'/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/comp{comp_num1+1}_comp{comp_num2+1}'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for i in range(9):
        for j in range(9):



            pca_x[0][comp_num1] = first_component[i]
            pca_x[0][comp_num2] = second_component[j]
            #print(pca_x[0][0], pca_x[0][1])
            x_list.append([pca_x[0][comp_num1], pca_x[0][comp_num2]])

            pca_x_0 = transformer.inverse_transform(pca_x[0]).astype(np.float32)
            out = model.sample_np(pca_x_0.reshape(1,512))
            img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
            images.append(img)
            #fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
            #axs.imshow(img)
            img.save(f'{save_dir}/comp{comp_num1+1}={first_component[i]}_comp{comp_num2+1}={second_component[j]}.png')

    # 得られた9*9の画像をグリッド表示する
    fig, axs = plt.subplots(9, 9, figsize=(20,20), tight_layout='True')
    cnt = 80
    cnt_i = 8
    for i in range(9):
        cnt_j = 8
        print("i:",cnt_i)
        for j in range(9):

            # imagesの中身を配列の後ろから順番に表示していく
            axs[i][j].imshow(images[cnt])
            axs[i][j].axis('off')
            # タイトルをつける
            # axs[i][j].set_title(f'comp1={first_component[i]}_comp2={second_component[j]}', fontsize=10)
            print("j:",cnt_j)
            cnt -= 1
            cnt_j -= 1

    # 画像を保存する
    fig.savefig(f'{save_dir}/gridimage_comp{comp_num1+1},comp{comp_num2+1}.png')
        

In [ ]:
pca_x = param_read
pca_x[0] = 0


# Create an array with values ranging from -10 to 10, divided equally into 8 segments
first_component = np.linspace(-10, 10, 9)

# Create an array with values ranging from -8 to 8, divided equally into 8 segments
second_component = np.linspace(-8, 8, 9)
images = []
x_list = []
for i in range(9):
    for j in range(9):
        pca_x[0][0] = first_component[i]
        pca_x[0][1] = second_component[j]
        #print(pca_x[0][0], pca_x[0][1])
        x_list.append([pca_x[0][0], pca_x[0][1]])

        pca_x_0 = transformer.inverse_transform(pca_x[0]).astype(np.float32)
        out = model.sample_np(pca_x_0.reshape(1,512))
        img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
        images.append(img)
        #fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
        #axs.imshow(img)
        # img.save(f'/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/comp1={first_component[i]}_comp2={second_component[j]}.png')

# 得られた9*9の画像をグリッド表示する
fig, axs = plt.subplots(9, 9, figsize=(20,20), tight_layout='True')
for i in range(9):
    for j in range(9):
        print(x_list[i*9+j])
        # imagesの中身を配列の後ろから順番に表示していく
        axs[i][j].imshow(images[i*9+j])
        axs[i][j].axis('off')
        # タイトルをつける
        # axs[i][j].set_title(f'comp1={first_component[i]}_comp2={second_component[j]}', fontsize=10)

# 画像を保存する
fig.savefig('/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/gridimage_comp1=-10to10_comp2=-8to8.png')
        

In [ ]:
pca_x = param_read
pca_x[0] = 0

x_comp = 0
y_comp = 1

# Create an array with values ranging from -10 to 10, divided equally into 8 segments
first_component = np.linspace(-10, 10, 9)

# Create an array with values ranging from -8 to 8, divided equally into 8 segments
second_component = np.linspace(-8, 8, 9)
images = []
x_list = []
for i in range(9):
    for j in range(9):
        pca_x[0][x_comp] = first_component[i]
        pca_x[0][y_comp] = second_component[j]
        #print(pca_x[0][0], pca_x[0][1])
        x_list.append([pca_x[0][x_comp], pca_x[0][y_comp]])

        pca_x_0 = transformer.inverse_transform(pca_x[0]).astype(np.float32)
        out = model.sample_np(pca_x_0.reshape(1,512))
        img = Image.fromarray((out * 255).astype(np.uint8)).resize((256,256),Image.LANCZOS)
        images.append(img)
        # fig, axs = plt.subplots(1, 1, figsize=(10,10), tight_layout='True')
        # axs.imshow(img)
        # img.save(f'/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/comp1={first_component[i]}_comp2={second_component[j]}.png')

# # 得られた9*9の画像をグリッド表示する
fig, axs = plt.subplots(9, 9, figsize=(20,20), tight_layout='True')
cnt = 80
cnt_i = 8
for i in range(9):
    cnt_j = 8
    print("i:",cnt_i)
    for j in range(9):
        #print(x_list[cnt])
        # imagesの中身を配列の後ろから順番に表示していく
        axs[i][j].imshow(images[cnt])
        axs[i][j].axis('off')
        # タイトルをつける
        axs[i][j].set_title(f'comp{x_comp+1}={first_component[cnt_i]}_comp{y_comp+1}={second_component[j]}', fontsize=10)
        print("j:",cnt_j)
        cnt -= 1
        cnt_j -= 1
        

    
    cnt_i -= 1
    

# 画像を保存する
# fig.savefig(f'/home/taki/ganspace/gen_img_lat-mean-to-comp_-2to2sigma_batch64x4_310k-iter_fid=38-30437_compnum=5/gridimage_comp{x_comp+1}=-10to10_comp{y_comp+1}=-8to8_with-title.png')
        

In [ ]:
count = 80
for i in range(9):
    for j in range(9):
    
        print(count)
        count -= 1

In [ ]:
first_component = np.linspace(-10, 10, 8)
first_component

In [ ]:
second_component = np.linspace(-8, 8, 8)
second_component

## ペア画像作成

In [ ]:
for i in range(1000):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    ax[0].axes.xaxis.set_visible(False) # X軸を非表示に
    ax[0].axes.yaxis.set_visible(False) # Y軸を非表示に
    ax[1].axes.xaxis.set_visible(False) # X軸を非表示に
    ax[1].axes.yaxis.set_visible(False) # Y軸を非表示に
    ax[0].imshow(images[i*2]) # 画像を表示
    ax[1].imshow(images[i*2+1]) # 画像を表示
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.close()


    plt.savefig('questionnaire/question-20~20/Q{}.png'.format(i+1), format='png')

In [ ]:
row=200 # 行数
col=2 # 列数
fig, ax = plt.subplots(nrows=row, ncols=col,figsize=(20,row*4))

for i in range(row*col):
    _r= i//col
    _c= i%col
    ax[_r,_c].set_title(i+1600, fontsize=32, y=-0.4)
    ax[_r,_c].axes.xaxis.set_visible(False) # X軸を非表示に
    ax[_r,_c].axes.yaxis.set_visible(False) # Y軸を非表示に
    ax[_r,_c].imshow(images[i]) # 画像を表示
    
fig.savefig('questionnaire/question/pair1600-1999.svg', format='svg')